In [1]:
import numpy as np
from astropy.table import Table
from scipy.sparse import lil_matrix
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
import time

In [2]:
def jaccard(a,b):
    """
    Calculate Jaccard distance between two arrays.
    :param a: array
        array of neighbors
    :param b: array
        array of neighbors
    :return: Jaccard distance.
    """
    a = np.array(a, dtype='int')
    b = np.array(b, dtype='int')
    a = a[a > -1]
    b = b[b > -1]
    union = np.union1d(a, b)
    intersection = np.intersect1d(a, b)
    return 1.0 - len(intersection)*1.0 / len(union)


def iterator_dist(indices, min_k):
    """
    Generator for computing distance matrix.
    :param indices: 2d array
        array of arrays of neighbors
    :param min_k: int
        minimum number of shared neighbors
    """
    for n in range(len(indices)):
        for m in indices[n][indices[n] > n]:
            if len(np.intersect1d(indices[m], indices[n])) > min_k:
                dist = jaccard(indices[n], indices[m])
                yield (n, m, dist)


In [7]:
# load data from APOGEE
directory = '/home/boquan/Data/'
filename = 'allStar-l31c.2.fits' # download data from https://www.sdss.org/dr14/irspec/spectro_data/
table = Table.read(directory+filename)
table = table[table['GLAT'] > 15]  # a subsample of stars with galactic latitude > 15 degrees
elements = ['C_FE', 'CI_FE', 'N_FE', 'O_FE', 'NA_FE', 'MG_FE', 'AL_FE', 'SI_FE', 'P_FE', 'S_FE', 'K_FE', 'CA_FE',
            'TI_FE', 'TIII_FE', 'V_FE', 'CR_FE', 'MN_FE', 'FE_H', 'CO_FE', 'NI_FE']
df = table[elements+['VHELIO_AVG', 'APOGEE_ID', 'SNR', 'STARFLAG']].to_pandas()
df = df[df['STARFLAG'] == 0]
df['APOGEE_ID'] = df['APOGEE_ID'].str.decode('utf-8') # remove duplicates by keeping the highest SNR
df = df.sort_values('SNR', ascending=False)
df = df.drop_duplicates(subset='APOGEE_ID', keep='first')
df = df.mask(df == -9999)
df = df.dropna(subset=elements+['VHELIO_AVG']) 
print(f'{len(df)} stars left after dropping NaNs and applying the cut in b')

17638 stars left after dropping NaNs and applying the cut in b


In [8]:
# parameters
num_nbhrs = 50  # number of nearest neighbors to retrieve for each star in the initial data space
min_k = 10  # minimum number of shared neighbors between two stars to be connected
eps = 0.35  # search radius parameter in DBSCAN
min_samples = 8  # minimum number of data points within the search radius to be considered a core point

In [9]:
# data matrix
data = df[elements].values

In [10]:
# get nearest neighbors
start = time.time()
nbrs = NearestNeighbors(n_neighbors=num_nbhrs, metric='manhattan').fit(data)
distances, indices = nbrs.kneighbors(data)
print('Nearest neighbors found. ')
print(f'Took {time.time() - start:.2f} seconds')

Nearest neighbors found. 
Took 10.63 seconds


In [11]:
# generate distance matrix
start = time.time()
S = lil_matrix((data.shape[0], data.shape[0]))
for (n, m, dist) in iterator_dist(indices, min_k):
    S[n, m] = dist
S += S.transpose()
print('Distance matrix created. ')
print(f'Took {time.time() - start:.2f} seconds')

Distance matrix created. 
Took 12.60 seconds


In [12]:
# DBSCAN clustering with precomputed distance matrix
start = time.time()
db = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed', n_jobs=-1).fit(S)
labels = db.labels_
n_clumps = max(labels)+1
print(f'{n_clumps} clusters found ({np.sum(labels > -1)} stars left)')
print(f'Took {time.time() - start:.2f} seconds')

3 clusters found (103 stars left)
Took 0.10 seconds
